In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# ✅ FIXED PATH
img_path = 'C:/Users/hp/Downloads/IMD/difference53.jpg'

# Check if path is valid
import os
if not os.path.isfile(img_path):
    print("Error: File does not exist at the given path.")
else:
    # Load image
    img = Image.open(img_path).convert('RGB')

    # Convert to grayscale
    gray_img = img.convert('L')

    # Apply threshold
    threshold = 50
    bw_img = gray_img.point(lambda x: 255 if x > threshold else 0, mode='1')

    # Save B/W image
    bw_path = 'C:/Users/hp/Downloads/IMD/image_bw_01.jpg'
    bw_img.save(bw_path)

    # Show result
    plt.imshow(bw_img, cmap='gray')
    plt.axis('off')
    plt.title("Black & White Image")
    plt.show()

    print(f"✅ Image processed and saved at: {bw_path}")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_red_columns(image, pixel_threshold=30, debug=False):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # HSV thresholds for red
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Count red pixels in each column
    red_pixel_counts_per_column = np.sum(red_mask > 0, axis=0)

    # Threshold to avoid counting faint/noisy columns
    red_columns = red_pixel_counts_per_column > pixel_threshold
    num_red_columns = np.sum(red_columns)

    if debug:
        # Plot red pixel count per column
        plt.figure(figsize=(12, 3))
        plt.plot(red_pixel_counts_per_column, color='red')
        plt.title("Red Pixel Count Per Column")
        plt.xlabel("Column Index")
        plt.ylabel("Red Pixels")
        plt.grid(True)
        plt.show()

    return num_red_columns, red_mask

# === STEP 1: Calibration from Reference Image ===
reference_image_path = 'C:/Users/kaurn/Desktop/500 45.png'  # Replace with your reference image
known_temperature = 45.0  # Red mercury section = 45°C

ref_img = cv2.imread(reference_image_path)
if ref_img is None:
    raise ValueError(f"Could not load reference image: {reference_image_path}")

ref_red_columns, ref_mask = detect_red_columns(ref_img, debug=True)
image_width = ref_img.shape[1]
TPP = known_temperature / ref_red_columns

print(f"\nREFERENCE CALIBRATION")
print(f"Total image width             : {image_width} pixels")
print(f"Red columns in reference     : {ref_red_columns}")
print(f"Known red temperature        : {known_temperature} °C")
print(f"Calculated TPP               : {TPP:.4f} °C per red column")

# === Visual Confirmation for Reference Image ===
plt.figure(figsize=(10, 3))
plt.imshow(cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB))
plt.title("Reference Image")
plt.axis('off')
plt.show()

plt.figure(figsize=(10, 3))
plt.imshow(ref_mask, cmap='gray')
plt.title("Detected Red Area (Reference)")
plt.axis('off')
plt.show()

# === STEP 2: Analyze Test Images ===
test_image_paths = [
    '500 70.jpg',  # Add more test images as needed
]

for test_path in test_image_paths:
    test_img = cv2.imread(test_path)
    if test_img is None:
        print(f"\nCould not load test image: {test_path}")
        continue

    test_red_columns, test_mask = detect_red_columns(test_img, debug=True)
    temperature = test_red_columns * TPP

    print(f"\nANALYSIS FOR: {test_path}")
    print(f"Red columns detected         : {test_red_columns}")
    print(f"Estimated Temperature        : {temperature:.2f} °C")

    # Visual display
    plt.figure(figsize=(10, 3))
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
    plt.title(f"{test_path} - Estimated Temp: {temperature:.2f} °C")
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(10, 3))
    plt.imshow(test_mask, cmap='gray')
    plt.title("Detected Red Area (Test Image)")
    plt.axis('off')
    plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_red_columns(image, pixel_threshold=30, debug=False):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # HSV thresholds for red
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Count red pixels in each column
    red_pixel_counts_per_column = np.sum(red_mask > 0, axis=0)

    # Threshold to avoid counting faint/noisy columns
    red_columns = red_pixel_counts_per_column > pixel_threshold
    num_red_columns = np.sum(red_columns)

    if debug:
        # Plot red pixel count per column
        plt.figure(figsize=(12, 3))
        plt.plot(red_pixel_counts_per_column, color='red')
        plt.title("Red Pixel Count Per Column")
        plt.xlabel("Column Index")
        plt.ylabel("Red Pixels")
        plt.grid(True)
        plt.show()

    return num_red_columns, red_mask

# === STEP 1: Calibration from Reference Image ===
reference_image_path = 'C:/Users/kaurn/Desktop/500 45.png'  # Replace with your reference image
known_temperature = 45.0  # Red mercury section = 45°C

ref_img = cv2.imread(reference_image_path)
if ref_img is None:
    raise ValueError(f" Could not load reference image: {reference_image_path}")

ref_red_columns, ref_mask = detect_red_columns(ref_img, debug=True)
image_width = ref_img.shape[1]
TPP = known_temperature / ref_red_columns

print(f"\n REFERENCE CALIBRATION")
print(f" Total image width             : {image_width} pixels")
print(f" Red columns in reference     : {ref_red_columns}")
print(f" Known red temperature        : {known_temperature} °C")
print(f" Calculated TPP               : {TPP:.4f} °C per red column")

# === Visual Confirmation for Reference Image ===
plt.figure(figsize=(10, 3))
plt.imshow(cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB))
plt.title("Reference Image")
plt.axis('off')
plt.show()

plt.figure(figsize=(10, 3))
plt.imshow(ref_mask, cmap='gray')
plt.title("Detected Red Area (Reference)")
plt.axis('off')
plt.show()

# === STEP 2: Analyze Test Images ===
test_image_paths = [
    '500 70.jpg',
    
]

for test_path in test_image_paths:
    test_img = cv2.imread(test_path)
    if test_img is None:
        print(f"\n Could not load test image: {test_path}")
        continue

    test_red_columns, test_mask = detect_red_columns(test_img, debug=True)
    temperature = test_red_columns * TPP

    print(f"\n ANALYSIS FOR: {test_path}")
    print(f" Red columns detected         : {test_red_columns}")
    print(f" Estimated Temperature        : {temperature:.2f} °C")

    # Visual display
    plt.figure(figsize=(10, 3))
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
    plt.title(f"{test_path} - Estimated Temp: {temperature:.2f} °C")
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(10, 3))
    plt.imshow(test_mask, cmap='gray')
    plt.title("Detected Red Area (Test Image)")
    plt.axis('off')
    plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# === Inputs ===
image_path = input("Enter full path to mercury image: ")
known_total_temp = 100  # Max temperature on the scale

# === Load and Validate ===
img = cv2.imread(image_path)
if img is None:
    print("Error: Image not found.")
    exit()

# === Detect Red Mercury Region ===
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v = hsv[:, :, 2]
_, brightness_mask = cv2.threshold(v, 50, 255, cv2.THRESH_BINARY)

# Red hue masks
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Final red mercury mask
final_mask = cv2.bitwise_and(red_mask, brightness_mask)

# === Bounding Box & Temperature ===
coords = cv2.findNonZero(final_mask)
if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    mercury_width = max_x - min_x + 1
    scale_width = img.shape[1]  # ✅ use full image width as scale
    temp_per_pixel = known_total_temp / scale_width
    measured_temp = mercury_width * temp_per_pixel

    # === Output ===
    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")

    # === Visualize ===
    boxed = img.copy()
    cv2.rectangle(boxed, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Detected Mercury")
    plt.show()
else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Input
image_path = input("Enter full path to mercury image: ")
known_total_temp = 100  # Set thermometer's max temp

# Load image
img = cv2.imread(image_path)
if img is None:
    print("Error: Image not found.")
    exit()

# Convert to HSV and apply brightness threshold
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v_channel = hsv[:, :, 2]
_, thresh_mask = cv2.threshold(v_channel, 50, 255, cv2.THRESH_BINARY)

# Red color detection in HSV
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Combine red detection with brightness
final_mask = cv2.bitwise_and(red_mask, thresh_mask)

# Detect red mercury region
coords = cv2.findNonZero(final_mask)
if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])
    mercury_width = max_x - min_x + 1
    scale_width = max_x + 1

    temp_per_pixel = known_total_temp / scale_width
    measured_temp = mercury_width * temp_per_pixel

    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")

    # Optional visualization
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.title("Detected Mercury Region")
    plt.axis('off')
    plt.show()

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# === Input ===
image_path = input("Enter full path to mercury image: ")
known_total_temp = 100  # Set the thermometer's max temp range

# === Read image ===
img = cv2.imread(image_path)
if img is None:
    print("Error: Image not found.")
    exit()

# === HSV + brightness threshold ===
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v_channel = hsv[:, :, 2]
_, thresh_mask = cv2.threshold(v_channel, 50, 255, cv2.THRESH_BINARY)

# Red mask range
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Combine brightness + red masks
final_mask = cv2.bitwise_and(red_mask, thresh_mask)

# Detect red mercury region
coords = cv2.findNonZero(final_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])
    mercury_width = max_x - min_x + 1

    # 🧠 Dynamically use actual thermometer width (e.g., 100px = 100°C)
    scale_width = img.shape[1]  # Or use max_x + 1 if full mercury is visible
    temp_per_pixel = known_total_temp / scale_width
    measured_temp = mercury_width * temp_per_pixel

    # Bounding box visualization
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.title("Detected Mercury Region")
    plt.axis('off')
    plt.show()

    # Final Output
    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# === Input ===
image_path = input("Enter full path to mercury image: ")
known_total_temp = 100  # Set the thermometer's max temp range

# === Read image ===
img = cv2.imread(image_path)
if img is None:
    print("Error: Image not found.")
    exit()

# === HSV conversion and red detection ===
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v_channel = hsv[:, :, 2]
_, thresh_mask = cv2.threshold(v_channel, 50, 255, cv2.THRESH_BINARY)

# Red color mask in HSV
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Combine with brightness mask to avoid dark reds
final_mask = cv2.bitwise_and(red_mask, thresh_mask)

# Detect red mercury region
coords = cv2.findNonZero(final_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])
    mercury_width = max_x - min_x + 1
    img_width = img.shape[1]

    # Temperature calculation
    temp_per_pixel = known_total_temp / img_width
    measured_temp = mercury_width * temp_per_pixel

    # Bounding box visualization
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.title("Detected Mercury Region")
    plt.axis('off')
    plt.show()

    # Final Output
    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# === User Input ===
image_path = input("Enter full path to mercury image: ")
known_total_temp = 100  # Total temperature scale (e.g., 100°C)

# === Load Image ===
img = cv2.imread(image_path)
if img is None:
    print("Error: Image not found.")
    exit()

# === Convert to HSV and extract brightness ===
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v_channel = hsv[:, :, 2]
_, thresh_mask = cv2.threshold(v_channel, 50, 255, cv2.THRESH_BINARY)

# === Red color masking (two ranges in HSV) ===
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# === Combine red detection with brightness threshold ===
final_mask = cv2.bitwise_and(red_mask, thresh_mask)

# === Find mercury pixels ===
coords = cv2.findNonZero(final_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])
    
    # === Bounding Box Visualization ===
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.title('Detected Mercury Bounding Box')
    plt.axis('off')
    plt.show()

    # === Temperature Calculation ===
    mercury_width = max_x - min_x + 1
    img_width = img.shape[1]
    temp_per_pixel = known_total_temp / img_width
    measured_temp = mercury_width * temp_per_pixel
    error = abs(known_total_temp - measured_temp)

    # === Output ===
    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")
    print(f"Error: {error:.2f} °C")

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
img = cv2.imread('demo53.png')  # Change path if needed
if img is None:
    print("Image not found.")
    exit()

# Convert to HSV and extract Value channel
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v_channel = hsv[:, :, 2]

# Threshold on brightness to isolate red region from black/white scale
_, thresh_mask = cv2.threshold(v_channel, 50, 255, cv2.THRESH_BINARY)

# Red color HSV ranges
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Create red color mask
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Combine red mask with brightness threshold
final_mask = cv2.bitwise_and(red_mask, thresh_mask)

# Detect red mercury region
coords = cv2.findNonZero(final_mask)
if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected Mercury Bounding Box')
    plt.show()

    mercury_width = max_x - min_x + 1
    image_width = img.shape[1]
    total_temp = 100  # full scale of thermometer

    temp_per_pixel = total_temp / image_width
    measured_temp = mercury_width * temp_per_pixel
    error = abs(53 - measured_temp)  # assuming we know actual temp was 53°C

    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")
    print(f"Error: {error:.2f} °C")

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
img = cv2.imread('demo100(1).png')
if img is None:
    print("Image not found.")
    exit()

# Convert to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define HSV red range
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Create mask for red
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Find red mercury region
coords = cv2.findNonZero(red_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    # Draw rectangle for visual validation
    img_copy = img.copy()
    cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected Mercury Bounding Box')
    plt.show()

    mercury_width = max_x - min_x + 1
    total_image_width = img.shape[1]  # e.g. 600
    actual_temp = 100  # Known temperature of this image

    temp_per_pixel = actual_temp / mercury_width
    measured_temp = mercury_width * temp_per_pixel
    error = abs(actual_temp - measured_temp)

    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")
    print(f"Error: {error:.2f} °C")

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('demo100(1).png')
if img is None:
    print("Image not found.")
    exit()

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
v_channel = hsv[:, :, 2]
_, thresh_mask = cv2.threshold(v_channel, 50, 255, cv2.THRESH_BINARY)

lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

final_mask = cv2.bitwise_and(red_mask, thresh_mask)

coords = cv2.findNonZero(final_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected Mercury Bounding Box')
    plt.show()

    mercury_width = max_x - min_x + 1  # should be 100 in this case
    known_temp = 100
    temp_per_pixel = known_temp / mercury_width
    measured_temp = mercury_width * temp_per_pixel
    error = abs(known_temp - measured_temp)

    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")
    print(f"Error: {error:.2f} °C")

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# --- Step 1: Load image ---
img_path = 'demo100(1).png'  # Replace with your path
img = cv2.imread(img_path)
if img is None:
    print("Error: Image not found.")
    exit()

# --- Step 2: Convert to HSV and apply threshold on value channel (V) ---
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Apply threshold = 50 on the V (brightness) channel
v_channel = hsv[:, :, 2]
_, thresh_mask = cv2.threshold(v_channel, 50, 255, cv2.THRESH_BINARY)

# --- Step 3: Detect red color regions ---
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Combine red detection with brightness threshold
final_mask = cv2.bitwise_and(red_mask, thresh_mask)

# --- Step 4: Find red mercury region ---
coords = cv2.findNonZero(final_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    # --- Step 5: Draw bounding box ---
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.title('Detected Mercury Bounding Box')
    plt.axis('off')
    plt.show()

    # --- Step 6: Calculate Temperature ---
    total_temp = 100  # known full-scale temperature
    img_width = img.shape[1]
    temp_per_pixel = total_temp / img_width
    mercury_length = max_x - min_x + 1
    measured_temp = mercury_length * temp_per_pixel
    error = abs(total_temp - measured_temp)

    # --- Step 7: Print Results ---
    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")
    print(f"Error: {error:.2f} °C")
else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# --- Step 1: Load image ---
img_path = 'demo100(1).png'  # Replace with your path if needed
img = cv2.imread(img_path)
if img is None:
    print("Error: Image not found.")
    exit()

# --- Step 2: Convert to HSV and isolate red ---
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# --- Step 3: Find red region coordinates ---
coords = cv2.findNonZero(red_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    # --- Step 4: Draw bounding box ---
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.title('Detected Mercury Bounding Box')
    plt.axis('off')
    plt.show()

    # --- Step 5: Temperature Calculation ---
    total_temp = 100  # actual full thermometer temperature
    img_width = img.shape[1]
    temp_per_pixel = total_temp / img_width
    mercury_length = max_x - min_x + 1
    measured_temp = mercury_length * temp_per_pixel
    error = abs(total_temp - measured_temp)

    # --- Step 6: Output ---
    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")
    print(f"Error: {error:.2f} °C")
else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('C:/Users/hp/Downloads/demo100.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

plt.imshow(mask, cmap='gray')
plt.title('White Mercury Detection')
plt.axis('off')
plt.show()

coords = cv2.findNonZero(mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    img_boxed = img.copy()
    cv2.rectangle(img_boxed, (x, y), (x + w, y + h), (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img_boxed, cv2.COLOR_BGR2RGB))
    plt.title('Detected Mercury Bounding Box')
    plt.axis('off')
    plt.show()

    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")

    total_temp = 100
    img_width = img.shape[1]
    temp_per_pixel = total_temp / img_width
    mercury_length = max_x - min_x
    measured_temp = mercury_length * temp_per_pixel

    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")

else:
    print("No white mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
img = cv2.imread('C:/Users/hp/Downloads/demo100.png')

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold to detect white/bright regions (mercury)
_, mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

# Show mask
plt.imshow(mask, cmap='gray')
plt.title('White Mercury Detection')
plt.axis('off')
plt.show()

# Find white (mercury) pixel coordinates
coords = cv2.findNonZero(mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    # Draw bounding box
    img_boxed = img.copy()
    cv2.rectangle(img_boxed, (x, y), (x + w, y + h), (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img_boxed, cv2.COLOR_BGR2RGB))
    plt.title('Detected Mercury Bounding Box')
    plt.axis('off')
    plt.show()

    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")

    # Temperature calculation (corrected)
    total_temp = 100  # ✅ total thermometer range in °C
    img_width = img.shape[1]  # usually 600
    temp_per_pixel = total_temp / img_width
    mercury_length = max_x - min_x
    measured_temp = mercury_length * temp_per_pixel

    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")

else:
    print("No white mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
img = cv2.imread('C:/Users/hp/Downloads/demo100.png')

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold to detect white/bright regions (mercury)
_, mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)  # Adjust 200 if needed

# Show mask
plt.imshow(mask, cmap='gray')
plt.title('White Mercury Detection')
plt.axis('off')
plt.show()

# Find white (mercury) pixel coordinates
coords = cv2.findNonZero(mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    # Draw bounding box
    img_boxed = img.copy()
    cv2.rectangle(img_boxed, (x, y), (x + w, y + h), (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img_boxed, cv2.COLOR_BGR2RGB))
    plt.title('Detected Mercury Bounding Box')
    plt.axis('off')
    plt.show()

    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")

    # Temperature calculation
    total_temp = 600  # total range in °C
    img_width = img.shape[1]
    temp_per_pixel = total_temp / img_width
    mercury_length = max_x - min_x
    measured_temp = mercury_length * temp_per_pixel

    print(f"Temperature per pixel: {temp_per_pixel:.2f}")
    print(f"Measured temperature: {measured_temp:.2f} °C")

else:
    print("No white mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read enhanced image
img_path = input("Enter full path to the enhanced mercury image: ")
img = cv2.imread(img_path)
if img is None:
    print("Error: Image not found. Check the path.")
    exit()

# Convert image to HSV for red detection
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define HSV range for red
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Red mask
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

plt.imshow(red_mask, cmap='gray')
plt.title('Binary Mercury Detection')
plt.axis('off')
plt.show()

# Find red pixel coordinates
coords = cv2.findNonZero(red_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected Mercury Region')
    plt.show()

    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    total_pixels = img.shape[1]     # image width
    temp_range = 100                # total temp in °C
    tpp = temp_range / total_pixels # temperature per pixel
    mercury_length = abs(max_x - 0)
    temperature = mercury_length * tpp

    print(f"Mercury bounding box:\nLeft: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")
    print(f"Temperature per pixel: {tpp:.2f}")
    print(f"Measured temperature: {temperature:.2f} °C")
else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Take image path as input from user
img_path = input("Enter full path to the mercury image: ")

# Load image
img = cv2.imread(img_path)
if img is None:
    print("Error: Image not found. Check the path.")
    exit()

# Convert image to HSV for red detection
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define HSV range for red (split into two parts)
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Create masks for both red ranges
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Show binary red detection mask
plt.imshow(red_mask, cmap='gray')
plt.title('Binary Mercury Detection')
plt.axis('off')
plt.show()

# Find non-zero (red) mercury points
coords = cv2.findNonZero(red_mask)

if coords is not None:
    # Get bounding box of detected red area
    x, y, w, h = cv2.boundingRect(coords)

    # Draw bounding box on original image
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show image with bounding box
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected Mercury Region')
    plt.show()

    # Mercury horizontal start and end
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")

    # Calibration info
    total_pixels = img.shape[1]  # Total image width
    temp_range = 100  # Total temperature range in °C
    pixels_per_degree = total_pixels / temp_range

    print(f"Pixels per °C: {pixels_per_degree:.2f}")

    # Calculate mercury length (from left side)
    mercury_length = abs(max_x - 0)
    temperature = mercury_length / pixels_per_degree

    print(f"Measured temperature: {temperature:.2f} °C")

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
img = cv2.imread('dummyImg.png')

# Convert image to HSV for red detection
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define HSV range for red (split into two parts)
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Create masks for both red ranges
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Show binary red detection mask
plt.imshow(red_mask, cmap='gray')
plt.title('Binary Mercury Detection')
plt.axis('off')
plt.show()

# Find non-zero (red) mercury points
coords = cv2.findNonZero(red_mask)

if coords is not None:
    # Get bounding box of detected red area
    x, y, w, h = cv2.boundingRect(coords)

    # Draw bounding box on original image
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show image with bounding box
    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected Mercury Region')
    plt.show()

    # Mercury horizontal start and end
    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")

    # Calibration info
    total_pixels = img.shape[1]  # Total image width
    temp_range = 100  # Total temperature range in °C
    pixels_per_degree = total_pixels / temp_range

    print(f"Pixels per °C: {pixels_per_degree:.2f}")

    # Calculate mercury length (from left side)
    mercury_length = abs(max_x - 0)
    temperature = mercury_length / pixels_per_degree

    print(f"Measured temperature: {temperature:.2f} °C")

else:
    print("No red mercury region detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Create image: 1 row x 100 pixels
img = np.zeros((1, 100, 3), dtype=np.uint8)

# First 20 pixels = red (mercury)
for i in range(20):
    img[0, i] = [0, 0, 255]  # Red in BGR

# Remaining pixels alternate black and white
for i in range(20, 100):
    img[0, i] = [255, 255, 255] if i % 2 == 0 else [0, 0, 0]

# Convert to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define red range in HSV (two ranges for red)
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Create masks for both red ranges
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
red_mask = cv2.bitwise_or(mask1, mask2)

# Show red mask
plt.imshow(red_mask, cmap='gray')
plt.title('Red Mercury Detection')
plt.axis('off')
plt.show()

# Find red mercury pixels
coords = cv2.findNonZero(red_mask)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 1)

    # Show final image with bounding box
    resized = cv2.resize(boxed_img, (1000, 100), interpolation=cv2.INTER_NEAREST)
    plt.imshow(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Detected Mercury (Only Red)")
    plt.show()

    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])
    mercury_length = max_x - min_x + 1

    total_pixels = img.shape[1]
    temperature_range = 600
    pixels_per_degree = total_pixels / temperature_range
    temperature = mercury_length / pixels_per_degree

    print(f"Red mercury from column {min_x} to {max_x}")
    print(f"Mercury pixel length: {mercury_length}")
    print(f"Measured temperature: {temperature:.2f} °C")

else:
    print("No red mercury detected.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Read image from folder
img_path = 'C:/Users/hp/Downloads/thermo_test.jpg'  # <-- change this to your image path
img = cv2.imread(img_path)

if img is None:
    print("Image not found. Check the path.")
    exit()

# Step 2: Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Step 3: Apply threshold
_, binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

# Step 4: Show binary detection
plt.imshow(binary, cmap='gray')
plt.title('Binary Mercury Detection')
plt.axis('off')
plt.show()

# Step 5: Detect non-zero pixels
coords = cv2.findNonZero(binary)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.title('Detected Mercury')
    plt.axis('off')
    plt.show()

    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")

    scale_left_pixel = 0
    scale_right_pixel = img.shape[1] - 1
    temp_range = 25 - 4
    pixel_distance = abs(scale_right_pixel - scale_left_pixel)
    pixels_per_degree = pixel_distance / temp_range

    mercury_length = abs(max_x - scale_left_pixel)
    temperature = 25 - (mercury_length / pixels_per_degree)

    print(f"Pixels per °C: {pixels_per_degree:.2f}")
    print(f"Measured temperature: {temperature:.2f} °C")

else:
    print("No region detected with given threshold.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Create the test image (1 row x 100 cols)
image = np.zeros((1, 100, 3), dtype=np.uint8)

# First 20% pixels are red (mercury)
for i in range(20):
    image[0, i] = [0, 0, 255]  # Red in BGR

# Remaining 80% alternate black and white
for i in range(20, 100):
    image[0, i] = [255, 255, 255] if i % 2 == 0 else [0, 0, 0]

# Step 2: Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 3: Apply threshold
_, binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

# Step 4: Display binary image
plt.imshow(binary, cmap='gray')
plt.title('Binary Mercury Detection')
plt.axis('off')
plt.show()

# Step 5: Find white (non-zero) pixels
coords = cv2.findNonZero(binary)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    boxed_img = image.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 255, 0), 1)

    # Resize for visibility
    boxed_img_resized = cv2.resize(boxed_img, (1000, 100), interpolation=cv2.INTER_NEAREST)
    plt.imshow(cv2.cvtColor(boxed_img_resized, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Width (columns): {w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")

    scale_left_pixel = 0      # +25°C
    scale_right_pixel = 99    # +4°C
    temp_range = 25 - 4       # = 21°C
    pixel_distance = abs(scale_right_pixel - scale_left_pixel)
    pixels_per_degree = pixel_distance / temp_range

    mercury_length = abs(max_x - scale_left_pixel)
    temperature = 25 - (mercury_length / pixels_per_degree)

    print(f"Pixels per °C: {pixels_per_degree:.2f}")
    print(f"Measured temperature: {temperature:.2f} °C")

else:
    print("No region detected with given threshold.")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('C:/Users/hp/Downloads/difference53.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

plt.imshow(binary, cmap='gray')
plt.title('Binary Mercury Detection')
plt.axis('off')
plt.show()

coords = cv2.findNonZero(binary)

if coords is not None:
    x, y, w, h = cv2.boundingRect(coords)
    boxed_img = img.copy()
    cv2.rectangle(boxed_img, (x, y), (x + w, y + h), (0, 0, 255), 2)

    plt.imshow(cv2.cvtColor(boxed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    min_x = np.min(coords[:, 0, 0])
    max_x = np.max(coords[:, 0, 0])

    print(f"Mercury bounding box:")
    print(f"Left: {x}, Right: {x + w}")
    print(f"Width (columns): {w}")
    print(f"Detected mercury starts at column: {min_x}")
    print(f"Detected mercury ends at column: {max_x}")

    scale_left_pixel = 0
    scale_right_pixel = img.shape[1] - 1

    temp_range = 25 - 4
    pixel_distance = abs(scale_right_pixel - scale_left_pixel)
    pixels_per_degree = pixel_distance / temp_range

    print(f"Pixels per °C: {pixels_per_degree:.2f}")

    mercury_length = abs(max_x - scale_left_pixel)
    temperature = 25 - (mercury_length / pixels_per_degree)

    print(f"Measured temperature: {temperature:.2f} °C")

else:
    print("No region detected with given threshold.")
